# Model4: Comments classification using BERT

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data preparation

In [16]:
df1 = pd.read_csv('../../data/semeval2017_twitter/combined_sentiment_dataset.csv')
df2 = pd.read_csv('../../data/YouTube_Comments_Dataset/YoutubeCommentsDataSet.csv')

In [17]:
df1.head()

,text,label,split,sentiment
0,"""QT @user In the original draft of the 7th boo...",2,train,positive
1,"""Ben Smith / Smith (concussion) remains out of...",1,train,neutral
2,Sorry bout the stream last night I crashed out...,1,train,neutral
3,Chase Headley's RBI double in the 8th inning o...,1,train,neutral
4,@user Alciato: Bee will invest 150 million in ...,2,train,positive


In [18]:
df1['label'].value_counts()

label
1    27479
2    21043
0    11377
Name: count, dtype: int64

2 - positive; 1 - neutral; 0 - negative

In [19]:
df1 = df1[["text", "label"]]
df1.columns = ["Comment", "Sentiment"]

In [20]:
df2.head()

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive


In [23]:
df2["Sentiment"] = df2["Sentiment"].replace({"negative": 0, "neutral": 1, "positive": 2}).infer_objects(copy=False)
df2["Sentiment"] = df2["Sentiment"].astype(int)

In [25]:
df = pd.concat([df1, df2], ignore_index=True)